In [1]:
import os
import nest_asyncio
from dotenv import load_dotenv
from typing import Optional

# --- Importaciones LlamaIndex ---
from llama_index.core import Settings, VectorStoreIndex, StorageContext, Document
from llama_index.core.node_parser import SemanticSplitterNodeParser
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.vector_stores.qdrant import QdrantVectorStore
from llama_parse import LlamaParse
#from llama_index.readers.file import PyMuPDFReade

# --- Importaciones Qdrant ---
from qdrant_client import QdrantClient

# --- Importaciones LangChain (Inteligencia de Extracción) ---
from langchain.chat_models import init_chat_model
from pydantic import BaseModel, Field

/home/david/uc_chile/proyecto_aplicado_1/rag/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 1. CARGA DE ENTORNO Y VALIDACIÓN (TU CÓDIGO)
# ==========================================================
nest_asyncio.apply()
load_dotenv()

# Configuración OpenAI
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    raise ValueError("❌ No se encontró OPENAI_API_KEY en el .env")

# Configuración Qdrant
QDRANT_URL = os.getenv("QDRANT_URL")
QDRANT_API_KEY = os.getenv("QDRANT_API_KEY")
COLLECTION_NAME = "metabolomics_agent_db" # Tu colección definida

if not QDRANT_URL:
    raise ValueError("❌ No se encontró QDRANT_URL en el .env")
if not QDRANT_API_KEY:
    raise ValueError("❌ No se encontró QDRANT_API_KEY en el .env")

# Configuración LlamaCloud
LLAMA_CLOUD_API_KEY = os.getenv("LLAMA_CLOUD_API_KEY")
if not LLAMA_CLOUD_API_KEY:
    raise ValueError("❌ No se encontró LLAMA_CLOUD_API_KEY en el .env")

print("✅ Credenciales validadas correctamente.")

✅ Credenciales validadas correctamente.


In [3]:
# 2. INICIALIZACIÓN DE CLIENTES Y MODELOS
# ==========================================================

# A. Cliente Qdrant (Conectado a tu Cloud)
qdrant_client = QdrantClient(
    url=QDRANT_URL, 
    api_key=QDRANT_API_KEY,
)

# Verificar conexión imprimiendo colecciones (opcional)
print(qdrant_client.get_collections())

# B. LlamaIndex Embeddings (Para Vectores Densos)
# Usará automáticamente la OPENAI_API_KEY del entorno
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")

# C. LangChain LLM (Para Extracción y CDE)
# Usando init_chat_model como pediste
llm_chat = init_chat_model(
    model="gpt-4o-mini",
    model_provider="openai",
    temperature=0
)

collections=[CollectionDescription(name='bio_semantic_db')]


In [4]:
# 3. SCHEMA DE DATOS (PYDANTIC)
# ==========================================================
class MetaboliteMetadata(BaseModel):
    """
    Modelo para la extracción estructurada de features metabolómicas.
    """
    mz_value: Optional[float] = Field(default=None, description="La relación masa/carga (m/z) exacta.")
    rt_value: Optional[float] = Field(default=None, description="El tiempo de retención (RT) en minutos.")
    compound_name: Optional[str] = Field(default=None, description="Nombre del compuesto químico identificado o putativo.")
    sample_source: Optional[str] = Field(default=None, description="Matriz biológica o fuente de la muestra (ej. Té Verde).")

# Vinculamos el LLM con el Schema
metadata_extractor = llm_chat.with_structured_output(MetaboliteMetadata)

In [5]:
# 4. FUNCIONES DEL PIPELINE (CDE + PARSE + EXTRACT)
# ==========================================================

async def procesar_pdf_completo(file_path):
    print(f"\n📄 [1/4] Parseando PDF: {file_path}")
    
    # A. LlamaParse: PDF -> Markdown
    parser = LlamaParse(result_type="markdown", language="es")
    documents = await parser.aload_data(file_path)
    full_text = documents[0].text
    
    # B. CDE (Contextual Document Embeddings - Simulado)
    # Generamos un resumen global para inyectar contexto a los chunks
    print("🧠 [2/4] Generando Contexto Global (CDE)...")
    prompt_cde = f"Resume en 2 oraciones el objetivo científico principal y la muestra analizada de: {full_text[:3000]}"
    resumen_global = llm_chat.invoke(prompt_cde).content
    print(f"      Contexto: {resumen_global}")

    # C. Semantic Chunking
    splitter = SemanticSplitterNodeParser(
        buffer_size=1, 
        breakpoint_percentile_threshold=95, 
        embed_model=Settings.embed_model
    )
    nodes = splitter.get_nodes_from_documents(documents)
    print(f"✂️ [3/4] Generados {len(nodes)} chunks semánticos.")

    # D. Enriquecimiento (Loop de Extracción)
    print("🔬 [4/4] Extrayendo m/z y RT...")
    nodes_enriquecidos = []
    
    for node in nodes:
        # Inyectar Contexto Global (CDE) en la metadata
        node.metadata["global_context"] = resumen_global
        node.metadata["file_name"] = os.path.basename(file_path) # Asegurar nombre de archivo
        
        try:
            # Extracción Estructurada con LangChain
            data = metadata_extractor.invoke(node.get_content())
            
            # Guardamos solo si hay datos valiosos
            if data.mz_value is not None:
                print(f"      ✅ Feature: m/z {data.mz_value} ({data.compound_name})")
                node.metadata.update(data.model_dump(exclude_none=True))
        except Exception:
            pass # Continuar si falla un chunk específico
            
        nodes_enriquecidos.append(node)

    return nodes_enriquecidos

In [6]:
import glob

# ==========================================================
# 5. EJECUCIÓN PRINCIPAL (CON RUTA RELATIVA CORREGIDA)
# ==========================================================

async def main_ingest():
    # 1. Detectar archivos en la carpeta '../data'
    # '..' significa subir un nivel (salir de notebooks)
    # 'data/*.pdf' significa entrar a data y buscar todo lo que termine en .pdf
    ruta_data = os.path.join("..", "data", "*.pdf")
    pdf_files = glob.glob(ruta_data)
    
    print(f"📂 Buscando archivos en: {ruta_data}")
    
    if not pdf_files:
        print("❌ No se encontraron PDFs en la carpeta '../data'. Verifica la ruta.")
        # Imprimir directorio actual para depurar
        print(f"   (Directorio actual del notebook: {os.getcwd()})")
        return None

    print(f"✅ Se encontraron {len(pdf_files)} archivos PDF: {[os.path.basename(f) for f in pdf_files]}")

    # 2. Conexión Vector Store -> Tu Cliente Qdrant
    vector_store = QdrantVectorStore(
        client=qdrant_client,           
        collection_name=COLLECTION_NAME,
        enable_hybrid=True,             
        batch_size=20
    )
    storage_context = StorageContext.from_defaults(vector_store=vector_store)

    all_nodes = []
    
    # 3. Procesamiento de Archivos encontrados
    for pdf in pdf_files:
        try:
            nodes = await procesar_pdf_completo(pdf)
            all_nodes.extend(nodes)
        except Exception as e:
            print(f"⚠️ Error procesando {os.path.basename(pdf)}: {e}")

    if not all_nodes:
        print("❌ No se generaron nodos válidos.")
        return None

    # 4. Indexación
    print("\n🚀 Subiendo vectores a Qdrant Cloud...")
    index = VectorStoreIndex(
        all_nodes,
        storage_context=storage_context,
        embed_model=Settings.embed_model
    )
    print(f"✅ ¡Ingesta completada! {len(all_nodes)} fragmentos subidos.")
    return index

# --- EJECUTAR ---
index = await main_ingest()

📂 Buscando archivos en: ../data/*.pdf
✅ Se encontraron 1 archivos PDF: ['1-s2.0-S0022316622152399-main.pdf']

📄 [1/4] Parseando PDF: ../data/1-s2.0-S0022316622152399-main.pdf
Started parsing the file under job_id f7e333c4-0935-4254-82f0-11b175401e8e
.🧠 [2/4] Generando Contexto Global (CDE)...
      Contexto: El objetivo principal del estudio fue investigar la absorción y el metabolismo de los antocianinas (ACN) en mujeres ancianas tras el consumo de extracto de saúco o arándano, identificando los metabolitos en la orina. La muestra analizada consistió en cuatro mujeres ancianas que consumieron 12 g de extracto de saúco y seis mujeres ancianas que ingirieron 189 g de arándano bajo, evaluando la excreción de ACN en orina y plasma.
✂️ [3/4] Generados 34 chunks semánticos.
🔬 [4/4] Extrayendo m/z y RT...
      ✅ Feature: m/z 287.1 (Cyanidin-3-glucoside)
      ✅ Feature: m/z 250.123 (Caffeine)
      ✅ Feature: m/z 1865.0 (None)
      ✅ Feature: m/z 0.0 (3-O-β-glucoside of pelargonidin)
     

## Simulación del retrieval

In [9]:
import os
from dotenv import load_dotenv
from qdrant_client import QdrantClient
from llama_index.vector_stores.qdrant import QdrantVectorStore
from llama_index.core.vector_stores import MetadataFilters, MetadataFilter, FilterOperator
from llama_index.core import VectorStoreIndex, Settings
from llama_index.embeddings.openai import OpenAIEmbedding

# 1. Cargar Credenciales
load_dotenv()

# Configurar el modelo de embedding (debe ser el mismo que usaste en la ingesta)
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")

# 2. Conectar a tu Base de Datos Existente
client = QdrantClient(
    url=os.getenv("QDRANT_URL"),
    api_key=os.getenv("QDRANT_API_KEY"),
)

vector_store = QdrantVectorStore(
    client=client,
    collection_name="metabolomics_agent_db", # <--- La misma colección que creaste
    enable_hybrid=True # <--- Importante para buscar por texto + semántica
)

# Reconstruimos el índice desde Qdrant (no lo crea de nuevo, solo se conecta)
index = VectorStoreIndex.from_vector_store(vector_store=vector_store)

# ==========================================
# 3. SIMULACIÓN DE CONSULTA DEL PROYECTO
# ==========================================

# 🔹 ESCENARIO: El usuario busca m/z 449.1
# En tu log vimos que detectó "Cyanidin-3-O-β-glucoside" con m/z 449.1
MZ_TARGET = 449.1 
TOLERANCIA = 0.5 # +/- 0.5 Da de margen de error

print(f"🔎 Buscando features con m/z entre {MZ_TARGET - TOLERANCIA} y {MZ_TARGET + TOLERANCIA}...")

# A. Definir Filtros (Solo traer documentos con ese rango de masa)
filters = MetadataFilters(
    filters=[
        MetadataFilter(
            key="mz_value", 
            operator=FilterOperator.GTE, 
            value=MZ_TARGET - TOLERANCIA
        ),
        MetadataFilter(
            key="mz_value", 
            operator=FilterOperator.LTE, 
            value=MZ_TARGET + TOLERANCIA
        ),
    ]
)

# B. Configurar el Motor de Recuperación (Retriever)
retriever = index.as_retriever(
    filters=filters,      # Aplicar filtro numérico primero
    similarity_top_k=3,   # Traer los 3 mejores chunks
    vector_store_kwargs={"qdrant_client": client}
)

# C. Ejecutar la Pregunta
# Preguntamos por "bioactividad" para usar la búsqueda híbrida (Semántica)
results = retriever.retrieve("¿Qué compuesto es y qué bioactividad tiene?")

# 4. Mostrar Resultados
if not results:
    print("❌ No se encontraron coincidencias.")
else:
    print(f"✅ Se encontraron {len(results)} documentos relevantes:\n")
    for i, res in enumerate(results):
        meta = res.metadata
        print(f"--- Resultado {i+1} ---")
        print(f"📄 Archivo: {meta.get('file_name')}")
        print(f"🧪 Feature Detectada: m/z {meta.get('mz_value')} | Nombre: {meta.get('compound_name')}")
        print(f"📝 Fragmento del texto:\n\"{res.get_content()[:200]}...\"\n")

🔎 Buscando features con m/z entre 448.6 y 449.6...
✅ Se encontraron 3 documentos relevantes:

--- Resultado 1 ---
📄 Archivo: 1-s2.0-S0022316622152399-main.pdf
🧪 Feature Detectada: m/z 449.1 | Nombre: Peonidin 3-glucoside
📝 Fragmento del texto:
"
ANTHOCYANIN METABOLITES IN HUMANS
lower detection limit under our current conditions would be 0.005 mg/L (based on cyanidin-3-glucoside).

# DISCUSSION

ACN have been considered to not be absorbed in..."

--- Resultado 2 ---
📄 Archivo: 1-s2.0-S0022316622152399-main.pdf
🧪 Feature Detectada: m/z 449.1 | Nombre: Peonidin-3-glucoside
📝 Fragmento del texto:
"Two possible pathways could describe the formation of this metabolite (Fig. 6). Cyanidin-3-glucoside could be absorbed intact and some methylated to form peonidin-3-glucoside in the liver (Fig. 6; Pat..."

--- Resultado 3 ---
📄 Archivo: 1-s2.0-S0022316622152399-main.pdf
🧪 Feature Detectada: m/z 449.1 | Nombre: Cyanidin-3-O-β-glucoside
📝 Fragmento del texto:
"The study protocol was approved by th

In [11]:
from langchain.messages import SystemMessage, HumanMessage

# ==========================================
# PASO 3: GENERACIÓN DE RESPUESTA (LLM)
# ==========================================

def generar_reporte_final(query_usuario, resultados_recuperados, llm):
    """
    Toma la pregunta del usuario y los chunks recuperados de Qdrant
    para generar una respuesta científica sintetizada.
    """
    
    # 1. Preparar el Contexto (Unir los fragmentos recuperados)
    contexto_texto = ""
    for i, res in enumerate(resultados_recuperados):
        meta = res.metadata
        contexto_texto += f"""
        [Documento {i+1}]
        - Fuente: {meta.get('file_name')}
        - Compuesto Detectado: {meta.get('compound_name')} (m/z {meta.get('mz_value')})
        - Fragmento: "{res.get_content()}"
        -------------------------------------------
        """

    # 2. Crear el Prompt para el Experto
    prompt_sistema = """
    Eres un asistente experto en Metabolómica y Química Analítica.
    Tu tarea es responder a la consulta del investigador basándote EXCLUSIVAMENTE en el contexto proporcionado.
    
    Reglas:
    1. Identifica el compuesto putativo basándote en el m/z y RT si están disponibles.
    2. Resume las bioactividades mencionadas en el texto recuperado.
    3. Cita la fuente (Documento X) para cada afirmación.
    4. Si hay varios candidatos con el mismo m/z (isómeros), menciónalos y explica la diferencia si el texto lo dice.
    5. Sé conciso y profesional.
    """

    prompt_usuario = f"""
    Consulta del Investigador: "{query_usuario}"
    
    Contexto Recuperado de la Base de Datos Interna:
    {contexto_texto}
    
    Por favor, genera el informe de anotación funcional:
    """

    # 3. Invocar al LLM (Usando tu modelo llm_chat de LangChain ya configurado)
    mensajes = [
        SystemMessage(content=prompt_sistema),
        HumanMessage(content=prompt_usuario)
    ]
    
    print("🤖 Generando respuesta final...")
    respuesta = llm.invoke(mensajes)
    
    return respuesta.content

# --- EJECUCIÓN ---
# Usamos los 'results' que obtuviste en la celda anterior
query = "¿Qué compuesto es y qué bioactividad tiene?"
informe_final = generar_reporte_final(query, results, llm_chat)

print("\n" + "="*50)
print("📝 INFORME FINAL DEL AGENTE RAG")
print("="*50)
print(informe_final)

🤖 Generando respuesta final...

📝 INFORME FINAL DEL AGENTE RAG
**Compuesto Putativo:**
El compuesto detectado es Peonidin 3-glucoside, con un m/z de 449.1. También se menciona Cyanidin-3-O-β-glucoside, que comparte el mismo m/z (449.1), pero se diferencia en su estructura química, ya que el primero es un derivado de peonidina y el segundo de cianidina. La diferencia radica en la sustitución de grupos en la estructura de los flavonoides (Documento 1, Documento 3).

**Bioactividad:**
1. Se ha discutido que los antocianos (ACN) como el Peonidin 3-glucoside pueden ser absorbidos en el cuerpo humano, lo que sugiere que podrían tener efectos bioactivos, aunque su absorción puede depender de la presencia de azúcares y otros factores (Documento 1).
2. Se ha propuesto que el Peonidin 3-glucoside podría ser metabolizado en el hígado a través de la formación de glucurónidos, lo que podría influir en su actividad biológica y en la eliminación de xenobióticos (Documento 2).

**Fuentes:**
- Document